### Load Test Data and Trained APG Sampler

In [1]:
import os
import torch
import numpy as np
from combinators.utils import save_models, load_models

from experiments.apgs_bshape.gibbs import gibbs_sweeps
from experiments.apgs_bshape.models import init_models

data_dir = './dataset/'
timesteps = 10
frame_pixels = 96
shape_pixels = 28
num_objects = 3

device = 'cuda:1'
num_epochs = 1000
lr = 2e-4
batch_size = 6
budget = 120
num_sweeps = 5

num_hidden_digit = 400
num_hidden_coor = 400
z_where_dim = 2
z_what_dim = 10


device = torch.device(device)
sample_size = budget // (num_sweeps + 1)
assert sample_size > 0, 'non-positive sample size =%d' % sample_size
mean_shape = torch.load(data_dir + 'mean_shape.pt').to(device)    
data_paths = []
for file in os.listdir(data_dir+'/video/'):
    if file.endswith('.pt') and \
    'timesteps=%d-objects=%d' % (timesteps, num_objects) in file:
        data_paths.append(os.path.join(data_dir+'/video/', file))
if len(data_paths) == 0:
    raise ValueError('Empty data path list.')

frames = torch.load(data_paths[0])[:,:timesteps]
frames_expand = frames.to(device).repeat(sample_size, 1, 1, 1, 1)

model_version = 'apg-timesteps=%d-objects=%d-sweeps=%d-samples=%d' % (timesteps, num_objects, num_sweeps, sample_size)
models = init_models(mean_shape=mean_shape, frame_pixels=frame_pixels, shape_pixels=shape_pixels, num_hidden_digit=num_hidden_digit, num_hidden_coor=num_hidden_coor, z_where_dim=z_where_dim, z_what_dim=z_what_dim, num_objects=num_objects, device=device)
load_models(models, 'cp-' + model_version)

apg = gibbs_sweeps(models, num_sweeps, timesteps)

out, frames = apg({"frames": frames_expand}, sample_dims=0, batch_dim=1, reparameterized=True), frames


RuntimeError: CUDA out of memory. Tried to allocate 1.38 GiB (GPU 1; 10.91 GiB total capacity; 9.59 GiB already allocated; 349.38 MiB free; 9.96 GiB reserved in total by PyTorch)

In [ ]:
from combinators import debug
if debug.runtime() == 'jupyter':
    from tqdm.notebook import trange, tqdm
else:
    from tqdm import trange, tqdm
from tqdm.contrib import tenumerate


In [ ]:
def get_samples(out, sweeps, T):
    recon_vals = out.trace['recon'].dist.probs
    z_where_vals = []
    for t in range(T):
        z_where_vals.append(out.trace['z_where_%d_%d'%(t,sweeps)].value.unsqueeze(2))
    z_where_vals = torch.cat(z_where_vals, 2)
    return (recon_vals.detach().cpu(), z_where_vals.detach().cpu())

In [ ]:
rs, ws = get_samples(out, num_sweeps, timesteps)

### Visualize Samples

In [2]:
from experiments.apgs_bshape.evaluation import viz_samples
viz_samples(frames, rs, ws, num_sweeps, num_objects, shape_pixels, fs=1)

ModuleNotFoundError: No module named 'experiments.apgs_bshape.evaluation'

### Computing log joint across all methods

### Computational Budget Analysis

### Comparison with Baselines